In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import re

In [2]:
# Cargamos los datos del excel
df = pd.read_excel("VT3.xlsx", sheet_name="Respuestas de formulario 1")

# Reemplazamos las valoraciones por valores numéricos y eliminamos (asignamos nan para filtrarlo después) las entradas del mismo alumno/a
df = df.replace('Excelente o muy por sobre lo esperado',5)
df = df.replace('Muy bueno',4)
df = df.replace('Satisfactorio o dentro de lo esperado',3)
df = df.replace('Regular',2)
df = df.replace('Insatisfactorio o  por debajo de lo esperado',1)

# Extraemos el correo e identificar a los evaluadores, colocando como llaves del diccionario el par (nombre estudiante, equipo) 
correos_dict = {}
for index, row in df.iterrows():
    for c in df.columns:
        if row[c] == "Este soy yo":
            correos_dict[(c.split(" [")[1][:-1].replace("].",""), row["Elige tu equipo"])] = row["Correo UC"]

df = df.replace('Este soy yo',np.nan)

# Solo consideramos la última respuesta del alumno
df = df.drop_duplicates(subset="Correo UC", keep="last")

# Obtenemos los distintos grupos
grupos = list(df["Elige tu equipo"].unique())

question_rename = None

# creamos este diccionario para almanacer el promedio de notas que le puso su equipo a un estudiante, para todos los 
# estudiantes del curso
notas_curso_dict = {}

# Iteramos por grupo
for grupo in grupos:
    # Filtramos por grupo
    df_grupo = df[df["Elige tu equipo"] == grupo]

    # Eliminamos las columnas asociadas a evaluaciones de alumnos fuera del grupo (sin respuestas)
    df_grupo = df_grupo.dropna(axis=1, how='all')

    # Obtenemos a los estudiantes del grupo
    estudiantes_grupo = []
    for c in df_grupo.columns[4:]:
        estudiante_grupo = c.split(" [")[1][:-1].replace("].","")
        if estudiante_grupo not in estudiantes_grupo:
            estudiantes_grupo.append(estudiante_grupo)
    estudiantes_grupo = sorted(list(set(estudiantes_grupo)))

    # Iteramos por alumno/a del grupo
    for estudiante_grupo in estudiantes_grupo:
        # Obtenemos las columnas con información relevante (las evaluaciones del alumno/a seleccionado/a)
        # En la lista relevant_values guardaremos las columnas en las que aparece el estudiante en el que estamos parados
        relevant_values = []
        for c in df_grupo.columns:
            estudiante_columna = re.search("\[(.*?)\]", c)
            if estudiante_columna is not None:
                estudiante_columna = estudiante_columna.group(1)
            if estudiante_columna == estudiante_grupo:
                relevant_values.append(c)

        # Transformamos los datos de string a un valor numérico (float)
        for c in df_grupo.columns[4:]:
            df_grupo[c] = df_grupo[df_grupo[c].notna()][c].astype(float)

        # Renombramos las preguntas para que queden más simples las llaves. En question_rename es una lista que tendrá
        # la información del nombre real de la pregunta y un índice creado que será asociado a esa pregunta (index_counter)
        question_rename = {}
        index_counter = 0
        for value in relevant_values:
            question_rename[value] = index_counter
            index_counter+=1

        # Guardamos en un diccionario la información de las notas del alumno/a
        notas_estudiante_dict = {question_rename[c]:round(df_grupo[c].mean(),1) for c in relevant_values}
        notas_curso_dict[(estudiante_grupo, grupo)] = notas_estudiante_dict


# Recuperamos el nombre completo de las preguntas y renombramos algunas llaves (que serán columnas)
inverse_question_rename = {v:k.split(" [")[0] for k,v in question_rename.items() if v != "Grupo"}
inverse_question_rename["Grupo"] = "Equipo"
inverse_question_rename["Correo UC"] = "Correo UC"
inverse_question_rename["Nombre Alumn@"] = "Nombre Alumn@"

# Identificamos los casos en donde no se tiene correo para un alumno/a (si no contestó la encuesta)
for k in notas_curso_dict.keys():
    if k in correos_dict.keys():
        notas_curso_dict[k]["Correo UC"] = correos_dict[k]
    else:
        notas_curso_dict[k]["Correo UC"] = "NO CONTESTÓ LA ENCUESTA"

# Agregamos la información del correo al diccionario con las notas
for k in notas_curso_dict.keys():
    aux_dict = {k2:v for k2,v in notas_curso_dict[k].items()}
    aux_dict["Grupo"] = k[1]
    aux_dict["Nombre Alumn@"] = k[0]
    notas_curso_dict[k] = aux_dict

# Pasamos el diccionario al formato de DataFrame y exportamos a excel
notas = pd.DataFrame.from_dict(data=notas_curso_dict,orient="index")
notas = notas.rename({c:inverse_question_rename[c] for c in notas.columns}, axis=1)
# Invertimos el orden de las columnas en el excel
notas.to_excel("Salida1.xlsx", encoding="latin-1", index=True, columns=notas.columns[::-1]) 
df_grupo = round(notas.groupby(['Equipo']).mean(),1)
# Invertimos el orden de las columnas en el excel
df_grupo.to_excel("Salida2.xlsx", encoding="latin-1", index=True, columns=df_grupo.columns[::-1]) 